In [1]:
import os
import pandas as pd
import subprocess
import xml.etree.ElementTree as ET
import re

In [7]:
#======================================================================
# 1. 사용자 설정
#======================================================================

# GT 정보가 담긴 Anvil 파일
anvil_file = './sample data/P01_R01.anvil'

# 클립을 생성할 원본 영상들이 있는 디렉토리
# (이전 단계에서 P01_R01.mp4를 V1~V4로 분할한 영상이 담긴 폴더)
source_video_dir = 'split_videos'

# 생성된 클립들을 저장할 최상위 디렉토리
output_clips_basedir = 'gt_clips'

# 파싱할 GT 트랙 이름 ("Action Label" 또는 "Meta-Action Label")
# "Action Label"이 더 세분화된 행동 정보를 포함합니다.
track_name_to_parse = "Meta-Action Label"

In [8]:
#======================================================================
# 2. Anvil 파일 파싱 함수 (사용자 코드 재활용)
#======================================================================

def parse_anvil_track(anvil_file, track_name):
    """
    지정된 .anvil 파일에서 특정 트랙 이름의 주석(annotation)을 파싱하여
    Pandas DataFrame으로 반환하는 함수.
    """
    if not os.path.exists(anvil_file):
        print(f"[오류] Anvil 파일 '{anvil_file}'을 찾을 수 없습니다.")
        return None

    try:
        tree = ET.parse(anvil_file)
        root = tree.getroot()
        annotations = []
        target_track = root.find(f'.//track[@name="{track_name}"]')

        if target_track is None:
            print(f"[정보] '{track_name}' 트랙을 찾을 수 없습니다.")
            return None

        for element in target_track.findall('el'):
            start_time = element.get('start')
            end_time = element.get('end')
            attribute = element.find('attribute')
            if attribute is not None and attribute.text:
                action_label = attribute.text
                annotations.append({
                    'Label': action_label,
                    'Start Time (s)': float(start_time),
                    'End Time (s)': float(end_time)
                })
        return pd.DataFrame(annotations)

    except ET.ParseError as e:
        print(f"[오류] XML 파일을 파싱하는 중 오류가 발생했습니다: {e}")
        return None
    except Exception as e:
        print(f"알 수 없는 오류가 발생했습니다: {e}")
        return None

In [9]:
#======================================================================
# 3. 메인 실행 로직
#======================================================================

# --- GT 데이터 로드 ---
print(f"'{anvil_file}'에서 '{track_name_to_parse}' 트랙 정보를 읽어옵니다...")
gt_df = parse_anvil_track(anvil_file, track_name_to_parse)

if gt_df is None or gt_df.empty:
    print("[실패] GT 데이터를 불러오지 못했습니다. 스크립트를 종료합니다.")
else:
    print(f"총 {len(gt_df)}개의 행동(클립)을 발견했습니다.")
    
    # --- 클리핑할 원본 영상 목록 가져오기 ---
    source_videos = sorted([f for f in os.listdir(source_video_dir) if f.endswith('.mp4')])
    if not source_videos:
        print(f"[오류] '{source_video_dir}' 폴더에 처리할 영상 파일이 없습니다.")
    else:
        print(f"다음 원본 영상들에 대해 클리핑을 시작합니다: {source_videos}")

'./sample data/P01_R01.anvil'에서 'Meta-Action Label' 트랙 정보를 읽어옵니다...
총 109개의 행동(클립)을 발견했습니다.
다음 원본 영상들에 대해 클리핑을 시작합니다: ['P01_R01_V1.mp4', 'P01_R01_V2.mp4', 'P01_R01_V3.mp4', 'P01_R01_V4.mp4']


In [10]:
gt_df

,Label,Start Time (s),End Time (s)
0,[2] Consult sheets,2.72000,9.00000
1,[7] Picking left,11.28000,12.84000
2,[12] Assemble system,13.84000,18.88000
3,[6] Picking in front,23.32000,25.44000
4,[12] Assemble system,26.48000,28.80000
...,...,...,...
104,[12] Assemble system,492.88000,497.88000
105,[4] Take screwdriver,504.64001,506.16000
106,[12] Assemble system,506.39999,509.79999
107,[5] Put down screwdriver,509.88000,512.08002


In [12]:
# --- 각 GT 항목에 대해 영상 클리핑 실행 ---
for index, row in gt_df.iterrows():
    label = row['Label']
    start_time = row['Start Time (s)']
    end_time = row['End Time (s)']
    duration = end_time - start_time

    # 파일명으로 사용할 수 없는 문자 제거 및 공백을 언더스코어로 변경
    sanitized_label = re.sub(r'[\\/*?:"<>|]', "", label).replace(' ', '_')
    
    # 클립을 저장할 하위 폴더 생성 (예: gt_clips/001_[OP010]_Consult_sheets/)
    clip_subdir_name = f"{index:03d}_{sanitized_label}"
    clip_output_path = os.path.join(output_clips_basedir, clip_subdir_name)
    os.makedirs(clip_output_path, exist_ok=True)
    
    print(f"\n[{index+1}/{len(gt_df)}] '{label}' 클립 생성 중...")
    print(f"  - 시간: {start_time:.2f}s ~ {end_time:.2f}s (길이: {duration:.2f}s)")
    print(f"  - 저장 위치: {clip_output_path}")

    # 각 뷰(V1~V4) 영상에 대해 클리핑 작업 수행
    for video_file in source_videos:
        input_path = os.path.join(source_video_dir, video_file)
        
        # 출력 파일명 설정 (예: clip_V1.mp4)
        view_name = os.path.splitext(video_file)[0].split('_')[-1] # V1, V2 등 추출
        output_filename = f"clip_{view_name}.mp4"
        output_path = os.path.join(clip_output_path, output_filename)

        # FFmpeg 명령어 생성
        # -c copy 옵션: 재인코딩 없이 원본 스트림을 그대로 복사하여 매우 빠릅니다.
        command = [
            'ffmpeg',
            '-y',                      # 덮어쓰기 허용
            '-i', input_path,          # 입력 파일
            '-ss', str(start_time),    # 시작 시간
            '-t', str(duration),       # 지속 시간
            # '-c', 'copy',              # 코덱 복사 (빠른 속도)
            output_path
        ]

        try:
            # FFmpeg 실행
            subprocess.run(command, check=True, capture_output=True, text=True)
        except FileNotFoundError:
            print("[치명적 오류] FFmpeg가 설치되어 있지 않거나 경로가 설정되지 않았습니다. 스크립트를 중단합니다.")
            exit()
        except subprocess.CalledProcessError as e:
            print(f"[오류] {video_file} 처리 중 FFmpeg 오류 발생:\n-> {e.stderr}")
            continue

print(f"\n🎉 모든 영상 클리핑 작업이 완료되었습니다!")
print(f"-> 최종 결과는 '{output_clips_basedir}' 폴더를 확인해주세요.")


[1/109] '[2] Consult sheets' 클립 생성 중...
  - 시간: 2.72s ~ 9.00s (길이: 6.28s)
  - 저장 위치: gt_clips/000_[2]_Consult_sheets

[2/109] '[7] Picking left' 클립 생성 중...
  - 시간: 11.28s ~ 12.84s (길이: 1.56s)
  - 저장 위치: gt_clips/001_[7]_Picking_left

[3/109] '[12] Assemble system' 클립 생성 중...
  - 시간: 13.84s ~ 18.88s (길이: 5.04s)
  - 저장 위치: gt_clips/002_[12]_Assemble_system

[4/109] '[6] Picking in front' 클립 생성 중...
  - 시간: 23.32s ~ 25.44s (길이: 2.12s)
  - 저장 위치: gt_clips/003_[6]_Picking_in_front

[5/109] '[12] Assemble system' 클립 생성 중...
  - 시간: 26.48s ~ 28.80s (길이: 2.32s)
  - 저장 위치: gt_clips/004_[12]_Assemble_system

[6/109] '[7] Picking left' 클립 생성 중...
  - 시간: 31.24s ~ 32.40s (길이: 1.16s)
  - 저장 위치: gt_clips/005_[7]_Picking_left

[7/109] '[8] Take measuring rod' 클립 생성 중...
  - 시간: 36.56s ~ 37.92s (길이: 1.36s)
  - 저장 위치: gt_clips/006_[8]_Take_measuring_rod

[8/109] '[4] Take screwdriver' 클립 생성 중...
  - 시간: 39.36s ~ 40.40s (길이: 1.04s)
  - 저장 위치: gt_clips/007_[4]_Take_screwdriver

[9/109] '[12] Assemble sy